In [5]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

c = 1.0
L = 2.0
J = 100
N = 100
h = L / J
x = np.linspace(0, L, J+1)

def initial_condition_function(x, cond_type):
    if cond_type == "Разрывная функция":
        u0 = np.where((x >= 0.4) & (x <= 0.6), 1.0, 0.0)
    elif cond_type == "Гауссова функция":
        u0 = np.exp(-((x - 0.5) ** 2) / (2 * 0.05**2))
    elif cond_type == "Обрезанный косинус":
        u0 = np.zeros_like(x)
        mask = (x >= 0.3) & (x <= 0.7)
        center = 0.5  
        half_width = 0.2  
        u0[mask] = np.cos((np.pi/2) * (x[mask] - center) / half_width)
        u0 = np.abs(u0)
    else:
        u0 = np.zeros_like(x)
    return u0

def solve_and_plot(sigma, time_index, ic_type):
    tau = sigma * h / c  
    T_effective = N * tau  
    u = np.zeros((N+1, J+1))
    u[0, :] = initial_condition_function(x, ic_type)
    
    for n in range(N):
        for j in range(1, J+1):
            u[n+1, j] = u[n, j] - sigma * (u[n, j] - u[n, j-1])
            
    t = time_index * tau
    x_shifted = (x - c * t) % L
    u_exact = initial_condition_function(x_shifted, ic_type)
    epsilon = np.linalg.norm(u[time_index, :] - u_exact)/np.linalg.norm(u[time_index, :])

    plt.figure(figsize=(8, 5))
    plt.plot(x, u[time_index, :], 'b-o', label=f'Численное, t = {t:.3f}')
    plt.plot(x, u_exact, 'r--', label='Теоретическое')
    plt.xlabel('x')
    plt.ylabel('u(x,t)')
    plt.title(f'σ = {sigma:.2f}, T = {T_effective:.3f}, ε = {epsilon:.3f}')
    plt.legend()
    plt.grid(True)
    plt.ylim(min(u[time_index, :].min(), u_exact.min()) - 0.1, max(u[time_index, :].max(), u_exact.max()) + 0.1)
    plt.show()

interact(solve_and_plot,
         sigma=FloatSlider(min=0.1, max=1.1, step=0.05, value=0.5, description="Число Куранта"),
         time_index=IntSlider(min=0, max=N, step=1, value=0, description="Временной слой"),
         ic_type=Dropdown(options=["Разрывная функция", "Гауссова функция", "Обрезанный косинус"],
                          value="Разрывная функция",
                          description="Начальное условие"));

interactive(children=(FloatSlider(value=0.5, description='Число Куранта', max=1.1, min=0.1, step=0.05), IntSli…

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

c = 1.0
L = 2.0
J = 100
N = 100
h = L / J
x = np.linspace(0, L, J+1)

def initial_condition_function(x, cond_type):
    if cond_type == "Разрывная функция":
        u0 = np.where((x >= 0.4) & (x <= 0.6), 1.0, 0.0)
    elif cond_type == "Гауссова функция":
        u0 = np.exp(-((x - 0.5) ** 2) / (2 * 0.05**2))
    elif cond_type == "Обрезанный косинус":
        u0 = np.zeros_like(x)
        mask = (x >= 0.3) & (x <= 0.7)
        center = 0.5  
        half_width = 0.2  
        u0[mask] = np.cos((np.pi/2) * (x[mask] - center) / half_width)
        u0 = np.abs(u0)
    else:
        u0 = np.zeros_like(x)
    return u0

def solve_and_plot(sigma, time_index, ic_type):
    tau = sigma * h / c  
    T_effective = N * tau  
    u = np.zeros((N+1, J+1))
    u[0, :] = initial_condition_function(x, ic_type)
    
    epsilon_values = []
    time_values = []
    
    for n in range(N):
        for j in range(1, J+1):
            u[n+1, j] = u[n, j] - sigma * (u[n, j] - u[n, j-1])
        
        t_current = n * tau
        x_shifted = (x - c * t_current) % L
        u_exact_current = initial_condition_function(x_shifted, ic_type)
        epsilon_current = np.linalg.norm(u[n, :] - u_exact_current) / np.linalg.norm(u[n, :])
        
        epsilon_values.append(epsilon_current)
        time_values.append(t_current)
    
    t = time_index * tau
    x_shifted = (x - c * t) % L
    u_exact = initial_condition_function(x_shifted, ic_type)

    fig, ax = plt.subplots(2, 1, figsize=(8, 10))
    
    ax[0].plot(x, u[time_index, :], 'b-o', label=f'Численное, t = {t:.3f}')
    ax[0].plot(x, u_exact, 'r--', label='Теоретическое')
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('u(x,t)')
    ax[0].set_title(f'Сравнение решений: σ = {sigma:.2f}, T = {T_effective:.3f}')
    ax[0].legend()
    ax[0].grid(True)
    ax[0].set_ylim(min(u[time_index, :].min(), u_exact.min()) - 0.1,
                   max(u[time_index, :].max(), u_exact.max()) + 0.1)
    
    ax[1].plot(time_values[:time_index+1], epsilon_values[:time_index+1], 'm-', label='Критерий сходимости ε')
    ax[1].set_xlabel('Время t')
    ax[1].set_ylabel('ε(t)')
    ax[1].set_title('Зависимость критерия сходимости от времени')
    ax[1].grid(True)
    ax[1].legend()
    
    plt.tight_layout()
    plt.show()

interact(solve_and_plot,
         sigma=FloatSlider(min=0.1, max=1.1, step=0.05, value=0.5, description="Число Куранта"),
         time_index=IntSlider(min=0, max=N, step=1, value=0, description="Временной слой"),
         ic_type=Dropdown(options=["Разрывная функция", "Гауссова функция", "Обрезанный косинус"],
                          value="Разрывная функция",
                          description="Начальное условие"));


interactive(children=(FloatSlider(value=0.5, description='Число Куранта', max=1.1, min=0.1, step=0.05), IntSli…

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

c = 1.0
L = 2.0
J = 100
N = 100
h = L / J
x = np.linspace(0, L, J+1)

def initial_condition_function(x, cond_type):
    if cond_type == "Разрывная функция":
        u0 = np.where((x >= 0.1) & (x <= 0.6), 1.0, 0.0)
    elif cond_type == "Гауссова функция":
        u0 = np.exp(-((x - 0.5) ** 2) / (2 * 0.05**2))
    elif cond_type == "Обрезанный косинус":
        u0 = np.zeros_like(x)
        mask = (x >= 0.3) & (x <= 0.7)
        center = 0.5  
        half_width = 0.2  
        u0[mask] = np.cos((np.pi/2) * (x[mask] - center) / half_width)
        u0 = np.abs(u0)
    else:
        u0 = np.zeros_like(x)
    return u0

def solve_and_plot(sigma, time_index, ic_type):
    tau = sigma * h / c  
    T_effective = N * tau  
    
    u_simple = np.zeros((N+1, J+1))
    u_simple[0, :] = initial_condition_function(x, ic_type)
    
    u_lw = np.zeros((N+1, J+1))
    u_lw[0, :] = initial_condition_function(x, ic_type)

    epsilon_values_simple = []
    epsilon_values_lw = []
    time_values = []
    
    for n in range(N):
        for j in range(1, J+1):
            u_simple[n+1, j] = u_simple[n, j] - sigma * (u_simple[n, j] - u_simple[n, j-1])
        
        for j in range(1, J):
            flux_left = 0.5 * (u_lw[n, j] + u_lw[n, j-1]) - 0.5 * sigma * (u_lw[n, j] - u_lw[n, j-1])
            flux_right = 0.5 * (u_lw[n, j+1] + u_lw[n, j]) - 0.5 * sigma * (u_lw[n, j+1] - u_lw[n, j])
            u_lw[n+1, j] = u_lw[n, j] - sigma * (flux_right - flux_left)
        
        u_lw[n+1, 0] = u_lw[n+1, J]
        u_lw[n+1, J] = u_lw[n+1, 0]
        
        t_current = n * tau
        x_shifted = (x - c * t_current) % L
        u_exact_current = initial_condition_function(x_shifted, ic_type)
        
        epsilon_simple_current = np.linalg.norm(u_simple[n, :] - u_exact_current) / np.linalg.norm(u_exact_current)
        epsilon_lw_current = np.linalg.norm(u_lw[n, :] - u_exact_current) / np.linalg.norm(u_exact_current)
        
        epsilon_values_simple.append(epsilon_simple_current)
        epsilon_values_lw.append(epsilon_lw_current)
        time_values.append(t_current)
    
    t = time_index * tau
    x_shifted = (x - c * t) % L
    u_exact = initial_condition_function(x_shifted, ic_type)

    fig, ax = plt.subplots(2, 1, figsize=(10, 6))
    
    ax[0].plot(x, u_simple[time_index, :], 'b-o', label=f'Схема первого порядка')
    ax[0].plot(x, u_lw[time_index, :], 'g-s', label=f'Лакс-Вендроф')

    ax[0].plot(x, u_exact, 'r--', label='Теоретическое')
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('u(x,t)')
    ax[0].set_title(f'Сравнение решений: σ={sigma:.2f}, T={T_effective:.3f}')
    ax[0].legend()
    ax[0].grid(True)
    
    ax[1].plot(time_values[:time_index+1], epsilon_values_simple[:time_index+1], 'm-', label='Схема первого порядка')
    ax[1].plot(time_values[:time_index+1], epsilon_values_lw[:time_index+1], 'g-', label='Лакс-Вендроф')
    ax[1].set_xlabel('Время t')
    ax[1].set_ylabel('ε(t)')
    ax[1].set_title('Зависимость ошибки от времени')
    ax[1].legend()
    ax[1].grid(True)
    
    plt.tight_layout()
    plt.show()

interact(solve_and_plot,
         sigma=FloatSlider(min=0.1, max=1.1, step=0.05, value=0.5, description="Число Куранта"),
         time_index=IntSlider(min=0, max=N, step=1, value=0, description="Временной слой"),
         ic_type=Dropdown(options=["Разрывная функция", "Гауссова функция", "Обрезанный косинус"],
                          value="Разрывная функция",
                          description="Начальное условие"));


interactive(children=(FloatSlider(value=0.5, description='Число Куранта', max=1.1, min=0.1, step=0.05), IntSli…